In [9]:
import torch
#import gensim
#import spacy
import numpy as np

In [10]:
x = torch.Tensor(5, 3)
print(x)


1.00000e-26 *
  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
  5.1662  0.0000  0.0000
  0.0000  0.0000  0.0000
  6.3711  0.0000  0.0000
[torch.FloatTensor of size 5x3]



In [13]:
a=torch.rand(2,3)
a


 0.4707  0.1666  0.1409
 0.8782  0.9003  0.9618
[torch.FloatTensor of size 2x3]

In [14]:
b=torch.rand(2,3)
b


 0.6680  0.4474  0.3186
 0.8957  0.3865  0.2950
[torch.FloatTensor of size 2x3]

In [15]:
a+b


 1.1387  0.6140  0.4595
 1.7739  1.2868  1.2568
[torch.FloatTensor of size 2x3]

In [16]:
torch.add(a,b)


 1.1387  0.6140  0.4595
 1.7739  1.2868  1.2568
[torch.FloatTensor of size 2x3]

In [19]:
# Giving an output tensor
res = torch.Tensor(2,3)
torch.add(a,b, out=res)
res


 1.1387  0.6140  0.4595
 1.7739  1.2868  1.2568
[torch.FloatTensor of size 2x3]

In [21]:
# addition in place
b.add_(a)


 1.1387  0.6140  0.4595
 1.7739  1.2868  1.2568
[torch.FloatTensor of size 2x3]

Any operation that mutates a tensor in-place is post-fixed with an _ For example: x.copy_(y), x.t_(), will change x.  
can use standard numpy-like indexing

In [24]:
# converting torch tensor to numpy array
p = torch.ones(3)
type(p)
print(p)


 1
 1
 1
[torch.FloatTensor of size 3]



In [23]:
q = p.numpy()
type(q)

numpy.ndarray

In [32]:
# let's change the p value
p.add_(1)
print(p)
print(q)  # this should also be change


 5
 5
 5
[torch.FloatTensor of size 3]

[ 1.  1.  1.]


In [33]:
d = np.ones(4)
e = torch.from_numpy(d)
np.add(d, 3, out=d)
print(d)
print(e)

[ 4.  4.  4.  4.]

 4
 4
 4
 4
[torch.DoubleTensor of size 4]



#### autograd

In [35]:
x = torch.autograd.Variable(torch.ones(2,2), requires_grad=True) 
print(x)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



In [36]:
y = x+2
y

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]

In [37]:
print(y.grad_fn)

In [38]:
z = y*y*3  
out = z.mean()
print(z, out)

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]



### Gradients

In [39]:
out.backward()  # same as out.backward(torch.Tensor([1.0]))

$ z_i = 3(x_i + 2)^2 $  
$ out = \frac{1}{4}\sum z_i $  
$ \frac{\mathrm{d}out}{{d}x} = \frac{3(x_i + 2)}{2}  $  

hence  

$ \frac{\mathrm{d}out}{{d}x}|x_i=1 = 4.5 $ 

In [40]:
# print out d(out)/dx
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



In [42]:
x = torch.randn(3)
x = torch.autograd.Variable(x, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

Variable containing:
 601.9485
 888.0287
 445.7296
[torch.FloatTensor of size 3]



In [43]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

Variable containing:
  102.4000
 1024.0000
    0.1024
[torch.FloatTensor of size 3]



### Neural Network

In [44]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)
